# Project: Wrangle and analyze 'WeRateDogs' Twitter Data 

## Introduction:

>

**Step 1:** First, all the necessary packages for wrangling, analyzing and visualizing data must be imported.

In [3]:
#import necessary packages
import numpy as np
import pandas as pd
import requests
import tweepy
import matplotlib.pyplot as plt
import seaborn as sns

#display visualizations in this notebook
%matplotlib inline

#format all visualization backgrounds with seaborn
sns.set()


 ## Part I: Gather Data

**Step 1:** The `twitter_archive_enhanced.csv` file onhand is read into a pandas dataframe.

**Step 2:** Now, the Udacity hosted file: `image_predictions.tsv` is downloaded via the requests library and assigned to a pandas dataframe.

**Step 3:** Use the Tweepy library to download each tweets JSON data into the single text file: `tweet_json.txt`. Then read the information on each line of the file into a pandas dataframe.

In [4]:
#assign twitter developer keys, authorze them, then assign twitter API to the api object variable
consumer_key = 'SP2BQjxXdQO2Z9b63y1pnbG56'
consumer_secret = '7NA3xssg2dpKOUhnCSXCvqJvVCJca5Mr9E6zcdwu1kxL86Ml8N'
access_token = '829356934478204928-UP3nvECGL7KwtPyvk6QsKZQrDPZct9A'
access_secret = 'PnFzZMNHunRW2oQItUkoTDmavcw2jES2ytvNXaunucR2p'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

## Part II: Assess Data

## Part III: Clean Data

## Part IV: Analyze and Visualize Data